In [1]:
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
def aceptar_cookies(año, liga, jornada):
    driver_dir = 'chrome_driver/chromedriver.exe'
    driver=webdriver.Chrome(driver_dir)
    driver.get("https://www.resultados-futbol.com/"+liga+str(año)+"/grupo1/jornada"+str(jornada))
    accept_cookies = driver.find_elements_by_xpath('//button[@class="sc-ifAKCX hYNOwJ"]')
    
    try:
        for button in accept_cookies:
            if button.text == "ACEPTO":
                relevant_button = button
                relevant_button.click()
    except:
        pass
    finally:
        return driver

In [3]:
def extraer_datos_jornada(soup):
    '''
    DOCSTRING
    '''
    if soup.find("table", {"id": 'tabla2'}):
        table_soup = soup.find("table", {"id": 'tabla2'}).find('tbody').find_all('tr')
    else:
        return None
    num_equipos = len(table_soup)

    Posicion = [table_soup[i].find('th').text for i in range(num_equipos)]
    Equipo = [table_soup[i].find('td', {'class':'equipo'}).find('a').text for i in range(num_equipos)]
    Puntos = [table_soup[i].find('td', {'class':'pts'}).text for i in range(num_equipos)]
    Jornada = [table_soup[i].find('td', {'class':'pj'}).text for i in range(num_equipos)]
    Ganados = [table_soup[i].find('td', {'class':'win'}).text for i in range(num_equipos)]
    Empatados = [table_soup[i].find('td', {'class':'draw'}).text for i in range(num_equipos)]
    Perdidos = [table_soup[i].find('td', {'class':'lose'}).text for i in range(num_equipos)]
    Goles_a_favor= [table_soup[i].find('td', {'class':'f'}).text for i in range(num_equipos)]
    Goles_en_contra= [table_soup[i].find('td', {'class':'c'}).text for i in range(num_equipos)]

    tabla_jornada = [Posicion, Equipo, Puntos, Jornada, Ganados, Empatados, Perdidos, Goles_a_favor, Goles_en_contra]

    # Nos aseguramos que ningun dato se ha saltado, y que hemos encontrado todos
    if len(set([len(i) for i in tabla_jornada])) == 1:
        return tabla_jornada
    else:
        return [None] * len(tabla_jornada)

In [4]:
def extraer_datos_partido(soup):
    '''
    DOCSTRING
    '''
    regex = re.compile('vevent')
    if soup.find("table", {"id": 'tabla1'}):
        table_soup = soup.find("table", {"id": 'tabla1'}).find('tbody').find_all("tr", {"class" : regex})
    else:
        return None
    
    num_partidos = len(table_soup)
    Equipo_Local = [table_soup[i].find('td', {'class':'equipo1'}).find_all('a')[1].text for i in range(num_partidos)]
    Equipo_Visitante = [table_soup[i].find('td', {'class':'equipo2'}).find_all('a')[1].text for i in range(num_partidos)]
    Resultado = [table_soup[i].find('td', {'class':'rstd'}).find('a', {'class':'url'}).find('span').text for i in range(num_partidos)]
    Fecha = [table_soup[i].find('td', {'class':'fecha'}).text for i in range(num_partidos)]

    tabla_partido = [Equipo_Local, Equipo_Visitante, Resultado, Fecha]

    # Nos aseguramos que ningun dato se ha saltado, y que hemos encontrado todos
    if len(set([len(i) for i in tabla_partido])) == 1:
        return tabla_partido
    else:
        return [None] * len(tabla_partido)

In [5]:
def extraer_num_jornadas(soup):
    jornadas_soup = soup.find("div", {"id":'desplega_jornadas', "class":'hidden'}).find_all('li')
    return len(jornadas_soup)

In [6]:
def extraccion_datos(año1, año2, *Ligas, primera_jornada = 1):
    '''
    DOCSTRING
    '''
    for liga in Ligas:
        for año in range(año1, año2 + 1):
            print(f'Extrayendo datos del año {año}') 
            driver = aceptar_cookies(año, liga, 1) # Accedemos simplemente a la primera jornada de ese año para comprobar cuantas jornadas hubo ese año
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')
            num_jornadas = extraer_num_jornadas(soup)
            driver.quit()
            for jornada in range(primera_jornada, num_jornadas + 1):
                print(f'\tExtrayendo jornada numero {jornada} del año {año}') 
                driver = aceptar_cookies(año, liga, jornada)
                page = driver.page_source
                soup = BeautifulSoup(page, 'html.parser')
                tabla_jornada = extraer_datos_jornada(soup)
                tabla_partido = extraer_datos_partido(soup)

                if tabla_jornada == None or tabla_partido == None: # Esto significaría que, a pesar de que la página diga que esa jornada tiene datos, en realidad durante ese año no hubo más jornadas. De esa manera, sale del 'for' de las jornadas, y pasa al siguiente año
                    print(f'------------------------------------------------------')
                    print(f'!!! La jornada {jornada} no existe en el año {año} !!!')
                    print(f'!!!            Saltando al siguiente año           !!!')
                    print(f'------------------------------------------------------')
                    driver.quit()
                    break
                
                for i, key in enumerate(list_jornadas[:-2]):
                    dict_jornadas[key].extend(tabla_jornada[i])

                dict_jornadas['Año'].extend([año] * len(tabla_jornada[0]))
                dict_jornadas['Liga'].extend([liga] * len(tabla_jornada[0]))

                for i, key in enumerate(list_partidos[:-3]):
                    dict_partidos[key].extend(tabla_partido[i])

                dict_partidos['Año_partido'].extend([año] * len(tabla_partido[0]))
                dict_partidos['Jornada_partido'].extend([jornada] * len(tabla_partido[0]))
                dict_partidos['Liga_partido'].extend([liga] * len(tabla_partido[0]))
               
                print(f'\tDatos extraídos, quedan {num_jornadas - jornada} jornadas restantes este año')
                driver.quit()
            print(f'Datos del año {año} extraídos, queda(n) {año2 - año - 1} año(s) restante(s)')
            primera_jornada = 1
    return list(Ligas)

In [7]:
list_jornadas = ['Posicion', 'Equipo', 'Puntos', 'Jornada', 'Ganados', 'Empatados', 'Perdidos', 'Goles_a_favor', 'Goles_en_contra', 'Año', 'Liga'] 
list_partidos = ['Equipo_Local', 'Equipo_Visitante', 'Resultado', 'Fecha', 'Año_partido', 'Jornada_partido', 'Liga_partido']
dict_jornadas = {x:[] for x in list_jornadas}
dict_partidos = {x:[] for x in list_partidos}

In [9]:
año_1 = 1995
año_2 = 2005
Ligas = extraccion_datos(año_1, año_2, 'premier')

Extrayendo datos del año 2005
	Extrayendo jornada numero 1 del año 2005
	Datos extraídos, quedan 37 jornadas restantes este año
	Extrayendo jornada numero 2 del año 2005
	Datos extraídos, quedan 36 jornadas restantes este año
	Extrayendo jornada numero 3 del año 2005
	Datos extraídos, quedan 35 jornadas restantes este año
	Extrayendo jornada numero 4 del año 2005
	Datos extraídos, quedan 34 jornadas restantes este año
	Extrayendo jornada numero 5 del año 2005
	Datos extraídos, quedan 33 jornadas restantes este año
	Extrayendo jornada numero 6 del año 2005
	Datos extraídos, quedan 32 jornadas restantes este año
	Extrayendo jornada numero 7 del año 2005
	Datos extraídos, quedan 31 jornadas restantes este año
	Extrayendo jornada numero 8 del año 2005
	Datos extraídos, quedan 30 jornadas restantes este año
	Extrayendo jornada numero 9 del año 2005
	Datos extraídos, quedan 29 jornadas restantes este año
	Extrayendo jornada numero 10 del año 2005
	Datos extraídos, quedan 28 jornadas restante

In [30]:
Data_Tabla = pd.DataFrame(dict_jornadas).drop_duplicates()
for liga in Ligas:
    Data_Tabla.to_excel(f"DDBB_Tabla_{año_1}_{año_2}_{liga}.xlsx")

Data_Partido = pd.DataFrame(dict_partidos).drop_duplicates()
Data_Partido.columns = ['Equipo_Local', 'Equipo_Visitante', 'Resultado', 'Fecha', 'Año', 'Jornada', 'Liga']
for liga in Ligas:
    Data_Tabla.to_excel(f"DDBB_Resultados_{año_1}_{año_2}_{liga}.xlsx")